The files we plan to use total about 8.65 gigabytes in file size. Our first step then, is to convert the larger JSON files into something more computationally efficient.

In [1]:
#!gsutil cp -r "gs://ba820-project/Yelp JSON" data
#!tar -xvf data/'Yelp JSON'/yelp_dataset.tar   Extracting tar file to retrieve JSONs.

In [1]:
import pandas as pd
business_df = pd.read_json("yelp_academic_dataset_business.json", lines = True)  # business.json is small, so no need to convert to parquet.

def slow_read(json_path):  # to keep our kernel from dying, we break the json into pieces to avoid overwhelming the VM. See AI appendix.
    dfs = []
    chunk_size = 100000
    for chunk in pd.read_json(json_path, lines=True, chunksize=chunk_size):
        dfs.append(chunk)
    new_df = pd.concat(dfs, ignore_index=True)
    return new_df
users_df = slow_read("yelp_academic_dataset_user.json")
reviews_df = slow_read("yelp_academic_dataset_review.json")
users_df.to_parquet("users.parquet")  # from this point forward in our analysis, we will use parquet files in place of the larger JSONs.
reviews_df.to_parquet("reviews.parquet")

Now the files we are using total to about 5 GBs in file size. This is much more manageable for our VM and will prevent it from crashing by simply loading it.

After that, it's time to explore the data and determine if anything needs further processing. First, the users file.

In [ ]:
import pandas as pd  # writing again to avoid running the above block
def df_basic_info(df):
    print(f'This dataframe has {df.shape[0]} rows.')
    print(f'This dataframe has {df.isnull().sum().sum()} null values.')
    return df.head(1)

users = pd.read_parquet("users.parquet")
df_basic_info(users)

In [ ]:
reviews = pd.read_parquet("reviews.parquet")
df_basic_info(reviews)

In [6]:
business = pd.read_json("yelp_academic_dataset_business.json", lines = True)
df_basic_info(business)

This dataframe has 150346 rows.
This dataframe has 37070 null values.


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None


The smallest file we will use is the only one with null values. Let's investigate further.

In [7]:
business.isnull().sum()

business_id         0
name                0
address             0
city                0
state               0
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      13744
categories        103
hours           23223
dtype: int64

It seems they are all within three features: attributes, categories, and hours.

Attributes refers to amenities that a business might have (Free WiFi, bathrooms, etc.).
Categories refers to common traits businesses on Yelp might have. This is used to group businesses.
Hours refers to the operating hours of the business.

None of these have a strong influence on the focus of our analysis: the content of reviews. However, categories could be helpful in subsetting businesses later in our analysis. Therefore, we will ignore these null values of attributes and hours. Regarding the 103 businesses missing a category, we will drop the rows in the business DataFrame and review DataFrame with corresponding business_id's.

In [8]:
null_bid = list(business[business["categories"].isna()]["business_id"])

In [9]:
print(f'The original dataframe has {reviews.shape[0]} rows.')
filtered_reviews = reviews[~reviews["business_id"].isin(null_bid)]
print(f'The new dataframe has {filtered_reviews.shape[0]} rows.')
print(f'{reviews.shape[0]-filtered_reviews.shape[0]} reviews were dropped.')

The original dataframe has 6990280 rows.
The new dataframe has 6989591 rows.
689 reviews were dropped.


In [10]:
filtered_business = business[~business["business_id"].isin(null_bid)]  # we drop 103 businesses out of 150346.

In [11]:
filtered_reviews.to_parquet("reviews2.parquet")

Another way we can filter down our files is by dropping rows in the business and user DataFrames with no reviews. These rows wouldn't be useful in our analysis and therefore are just taking up space.

In [12]:
filtered_business[filtered_business["review_count"]==0]  # there are no reviewless businesses.
filtered_business.to_parquet("business.parquet")

In [13]:
users = users[users["review_count"]!=0].copy()

In [14]:
print(f'{1987897 - users.shape[0]} users with no reviews were dropped.')

54 users with no reviews were dropped.


In [15]:
users.to_parquet("users2.parquet")

We'll use reviews2.parquet, users2.parquet, and business.parquet in our EDA and beyond.

In [16]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
